In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt


##########################################
###We implement our method on MNIST ######
# We apply the random search to update weights.
latent_dim = 8  # Dimensionality of the latent space

# Load MNIST dataset
(x_train, _), (x_test, _) = mnist.load_data()

# Normalize and reshape the data to include a channel dimension
x_train = np.expand_dims(x_train, -1).astype('float32') / 255.
x_test = np.expand_dims(x_test, -1).astype('float32') / 255.


encoder_inputs = layers.Input(shape=(28, 28, 1))
x = layers.Conv2D(32, 3, activation='relu', strides=2, padding='same')(encoder_inputs)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(64, 3, activation='relu', strides=2, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.25)(x)
z = layers.Dense(latent_dim, name='z')(x)
encoder = models.Model(encoder_inputs, z, name='encoder')
#encoder.summary()

latent_inputs = layers.Input(shape=(latent_dim,))
x = layers.Dense(7 * 7 * 64, activation='relu')(latent_inputs)
x = layers.Reshape((7, 7, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation='relu', strides=2, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2DTranspose(32, 3, activation='relu', strides=2, padding='same')(x)
x = layers.BatchNormalization()(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation='sigmoid', padding='same')(x)
decoder = models.Model(latent_inputs, decoder_outputs, name='decoder')
#decoder.summary()

In [4]:
# Access all layers in the encoder model
for layer in encoder.layers:
    print(f"Layer: {layer.name}")
    print(f"  Trainable: {layer.trainable}")
    print(f"  Layer Type: {type(layer)}")
    print(f"  Number of Trainable Weights: {len(layer.trainable_weights)}")
    print()


Layer: input_5
  Trainable: True
  Layer Type: <class 'tensorflow.python.keras.engine.input_layer.InputLayer'>
  Number of Trainable Weights: 0

Layer: conv2d_4
  Trainable: True
  Layer Type: <class 'tensorflow.python.keras.layers.convolutional.Conv2D'>
  Number of Trainable Weights: 2

Layer: batch_normalization_8
  Trainable: True
  Layer Type: <class 'tensorflow.python.keras.layers.normalization_v2.BatchNormalization'>
  Number of Trainable Weights: 2

Layer: conv2d_5
  Trainable: True
  Layer Type: <class 'tensorflow.python.keras.layers.convolutional.Conv2D'>
  Number of Trainable Weights: 2

Layer: batch_normalization_9
  Trainable: True
  Layer Type: <class 'tensorflow.python.keras.layers.normalization_v2.BatchNormalization'>
  Number of Trainable Weights: 2

Layer: flatten_2
  Trainable: True
  Layer Type: <class 'tensorflow.python.keras.layers.core.Flatten'>
  Number of Trainable Weights: 0

Layer: dense_4
  Trainable: True
  Layer Type: <class 'tensorflow.python.keras.layers.

In [6]:
for layer in encoder.layers:
    if layer.trainable_weights:
        print(f"Layer: {layer.name}")
        for weight in layer.trainable_weights:
            print(f"  Weight Name: {weight.name}")
            print(f"  Shape: {weight.shape}")
        print()


Layer: conv2d_4
  Weight Name: conv2d_4/kernel:0
  Shape: (3, 3, 1, 32)
  Weight Name: conv2d_4/bias:0
  Shape: (32,)

Layer: batch_normalization_8
  Weight Name: batch_normalization_8/gamma:0
  Shape: (32,)
  Weight Name: batch_normalization_8/beta:0
  Shape: (32,)

Layer: conv2d_5
  Weight Name: conv2d_5/kernel:0
  Shape: (3, 3, 32, 64)
  Weight Name: conv2d_5/bias:0
  Shape: (64,)

Layer: batch_normalization_9
  Weight Name: batch_normalization_9/gamma:0
  Shape: (64,)
  Weight Name: batch_normalization_9/beta:0
  Shape: (64,)

Layer: dense_4
  Weight Name: dense_4/kernel:0
  Shape: (3136, 64)
  Weight Name: dense_4/bias:0
  Shape: (64,)

Layer: z
  Weight Name: z/kernel:0
  Shape: (64, 8)
  Weight Name: z/bias:0
  Shape: (8,)



In [7]:
for layer in encoder.layers:
    if layer.trainable_weights:
        for weight in layer.trainable_weights:
            shape = weight.shape
            # For example, add Gaussian noise with mean 0 and stddev 0.01
            update_tensor = tf.random.normal(shape, mean=0.0, stddev=0.01)
            weight.assign_add(update_tensor)


In [8]:
import tensorflow as tf

# Iterate over all layers in the encoder
for layer in encoder.layers:
    # Check if the layer has trainable weights
    if layer.trainable_weights:
        print(f"Layer: {layer.name}")
        for weight in layer.trainable_weights:
            # Get weight name and shape
            weight_name = weight.name
            weight_shape = weight.shape
            print(f"  Weight: {weight_name}, Shape: {weight_shape}")

            # Create an update tensor of the same shape
            # Here, we use zeros, but you can use any tensor
            update_tensor = tf.zeros(weight_shape)

            # Update the weight
            weight.assign_add(update_tensor)

            print(f"  Updated weight: {weight_name}")
        print()


Layer: conv2d_4
  Weight: conv2d_4/kernel:0, Shape: (3, 3, 1, 32)
  Updated weight: conv2d_4/kernel:0
  Weight: conv2d_4/bias:0, Shape: (32,)
  Updated weight: conv2d_4/bias:0

Layer: batch_normalization_8
  Weight: batch_normalization_8/gamma:0, Shape: (32,)
  Updated weight: batch_normalization_8/gamma:0
  Weight: batch_normalization_8/beta:0, Shape: (32,)
  Updated weight: batch_normalization_8/beta:0

Layer: conv2d_5
  Weight: conv2d_5/kernel:0, Shape: (3, 3, 32, 64)
  Updated weight: conv2d_5/kernel:0
  Weight: conv2d_5/bias:0, Shape: (64,)
  Updated weight: conv2d_5/bias:0

Layer: batch_normalization_9
  Weight: batch_normalization_9/gamma:0, Shape: (64,)
  Updated weight: batch_normalization_9/gamma:0
  Weight: batch_normalization_9/beta:0, Shape: (64,)
  Updated weight: batch_normalization_9/beta:0

Layer: dense_4
  Weight: dense_4/kernel:0, Shape: (3136, 64)
  Updated weight: dense_4/kernel:0
  Weight: dense_4/bias:0, Shape: (64,)
  Updated weight: dense_4/bias:0

Layer: z
  W

In [13]:
import numpy as np

# Dictionary to hold the weights for each layer
layer_weights = {}

for layer in encoder.layers:
    if layer.trainable_weights:
        print(f"Layer: {layer.name}")
        weights = []
        for weight_var in layer.trainable_weights:
            # Convert the TensorFlow variable to a NumPy array
            weight_array = weight_var.numpy()
            weights.append(weight_array)
            print(f"  Weight: {weight_var.name}, Shape: {weight_array.shape}")
        # Store the weights in the dictionary
        layer_weights[layer.name] = weights
        print()


Layer: conv2d_4
  Weight: conv2d_4/kernel:0, Shape: (3, 3, 1, 32)
  Weight: conv2d_4/bias:0, Shape: (32,)

Layer: batch_normalization_8
  Weight: batch_normalization_8/gamma:0, Shape: (32,)
  Weight: batch_normalization_8/beta:0, Shape: (32,)

Layer: conv2d_5
  Weight: conv2d_5/kernel:0, Shape: (3, 3, 32, 64)
  Weight: conv2d_5/bias:0, Shape: (64,)

Layer: batch_normalization_9
  Weight: batch_normalization_9/gamma:0, Shape: (64,)
  Weight: batch_normalization_9/beta:0, Shape: (64,)

Layer: dense_4
  Weight: dense_4/kernel:0, Shape: (3136, 64)
  Weight: dense_4/bias:0, Shape: (64,)

Layer: z
  Weight: z/kernel:0, Shape: (64, 8)
  Weight: z/bias:0, Shape: (8,)



In [19]:
layer_weights['conv2d_4'][0].shape

(3, 3, 1, 32)